In [1]:
from datasets import load_dataset

ds = load_dataset("cfilt/iitb-english-hindi", split='train')
print(ds)

README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

Dataset({
    features: ['translation'],
    num_rows: 1659083
})


In [2]:
from itertools import chain

# Extract sentences
data = ds["translation"]
corpus = list(chain.from_iterable([[x["en"], x["hi"]] for x in data]))

# Optional: Save to a file (for large corpus)
with open("train.txt", "w", encoding="utf-8") as f:
    for line in corpus:
        f.write(line.strip() + "\n")

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers import normalizers
from tokenizers import pre_tokenizers

# Initialize tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence([Lowercase()])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Set up trainer
trainer = BpeTrainer(vocab_size=10000, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])

# Train
tokenizer.train(files=["train.txt"], trainer=trainer)

# Save the tokenizer
tokenizer.save("bpe_tokenizer.json")

In [4]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

encoded = tokenizer.encode("आप कैसे हैं? How are you ?")
print("Tokens:", encoded.tokens)
print("IDs:", encoded.ids)

Tokens: ['आप', 'कैसे', 'हैं', '?', 'how', 'are', 'you', '?']
IDs: [1113, 3750, 904, 37, 1904, 923, 892, 37]
